# Prototyping simulation study using DEC model for Cophylogeny

This notebook is a basic run through of the workflow that will be used to perform simulations for testing the usefulness of utilizing the epoch-DEC model used in biogeography for examining cophylogeny. Here the dispersal part of this model corresponds to host-switching, the extirpation part to extinction of a symbiont within a host, and the cladogenesis part the various sorting events occuring when the host speciates (including cospeciation). The hosts here act as the biogeographic areas upon which the symbionts live. For now I am testing this using only true trees simulated using [`treeducken`](https://github.com/wadedismukes/treeducken). While these simulations will reflect a mode of gene and spceies tree evolution and are thus not the most biologically realistic way to test this model, I think this is a good starting place. Especially with regards to comparing these models to other methods for examining cophylogeny. 


### Functions for making `treeducken` settings files

Below are a few functions used to make the settings files for treeducken. For now I am varying only transfer rate with 12 extant tips. The first of these functions `calculate_birth_rate` calculates the birth rate for a number of extant tips, given an expected tree depth, and turnover rate. This allows all of the simulation regimes to have roughly the same timescale upon which they act. The last two functions in this codeblock are simply utilities for generating the settings files. 

In [4]:
import math

def calculate_birth_rate(ntaxa, k, turnover, tre_dep):
    if(turnover != 0):
        sum_val_i = 0.0
        sum_val_j = 0.0
        birth_rate = ((k + 1) / tre_dep) * calculate_combinations(ntaxa, k + 1) * (-1)**k
        for i in range(0, ntaxa - k - 1):
            temp = calculate_combinations(ntaxa - k - 1, i)
            temp *= 1 / ((k + i + 1) * turnover)
            temp *= (1/turnover - 1)**(k + i)
            for j in range(1, k + i + 1):
                temp_j = calculate_combinations(k + i, j)
                temp_j *= ((-1)**j) / j
                temp_j *= (1 - (1 / (1 - turnover))**j)
                sum_val_j += temp_j
            sum_val_i += temp * (math.log(1 / (1 - turnover)) - sum_val_j)
        birth_rate *= sum_val_i  
    else:
        sum_val = 0.0
        for i in range(k + 1, ntaxa + 1):
            sum_val += (1 / (i * tre_dep))
        birth_rate = abs(2.0 * sum_val)
    return(birth_rate)


def calculate_combinations(n, k):
    n_fac = math.factorial(n)
    k_fac = math.factorial(k)
    n_k_fac = math.factorial(n - k)
    numer = math.log(n_fac)
    denom = math.log(k_fac) + math.log(n_k_fac)
    return(round(math.exp(numer - denom)))

def make_settings_dict(sbr, sdr, trr, gbr, gdr, nt, ipp, ne, reps, nl, ng):
    settings_dict = {}
    key_list = [("-sbr",sbr),
                ("-sdr",sdr),
                ("-gbr", gbr),
                ("-gdr", gdr),
                ("-lgtr",trr),
                ("-ipp",ipp),
                ("-nt",nt),
                ("-r",reps),
                ("-ne",ne),
                ("-nl",nl),
                ("-ng",ng),
                ("-o", "hsrate_" + str(trr)),
                ("-sout", 0)]
    settings_dict = dict(key_list)
    return(settings_dict)

def settings_writer(dictionary_settings, file_begin):
    file_name_end = "_settings.txt"
    file_begin += file_name_end       
    file_handle = open(file_begin, "w")
    for key, value in dictionary_settings.items():
        file_handle.write(key + " " + str(value) + "\n")
    file_handle.close()

### Setting simulation parameters and printing files
Here I have set the model parameters to be printed into settings files at the top. The for loop generates the number of regimes I want to simulate under. The number of tips and timing (13 here corresponding to 13 million years ago (mya)) were chosen to mimic the Geomyidae gophers and their chewing lice parasites.

In [5]:
nt = 12
turnover = 0.5
br = calculate_birth_rate(nt, 1, turnover, 13)
dr = br * turnover
trr = [0.0, 0.25, 0.5, 1.0]
gbr = 0.1
gdr = 0.1
reps = 1000
nl = 1
ng = 1
ipp = 1.0
ne = 1.0
num_setting_regimes = len(trr)

for i in range(num_setting_regimes):
    settings_regime_name = str(trr[i]) + "_trate"
    sett_dict = make_settings_dict(br, dr, trr[i], gbr, gdr, nt, 1, 1, reps, nl, ng)
    settings_writer(sett_dict, settings_regime_name)

## Running simulations
Now that I have printed out my settings files, I can use `treeducken` to generate my trees. First, I have to install `treeducken` here in the docker instance of this jupyter notebook.

In [6]:
%%bash
git clone https://github.com/wadedismukes/treeducken.git
cd treeducken/src && make install

g++ -o ../treeducken Treeducken.o SpeciesTree.o Simulator.o GeneTree.o LocusTree.o MbRandom.o Tree.o Engine.o


fatal: destination path 'treeducken' already exists and is not an empty directory.


Now run the simulations! 

In [7]:
%%bash

find *_settings.txt | xargs -I {} treeducken/treeducken -i {}

############################################################
####	treeducken, version 0.1 			####
####	e845a82c08ba308f75f94a270b81a36870299b94	####
############################################################
Simulating sets of three trees.
		output file name prefix         = hsrate_0.0
		Number of extant taxa           = 12
		Number of replicates            = 1000
		Number of loci to simulate      = 1
		Number of genes to simulate     = 1
		Species birth rate              = 10642.5
		Species death rate              = 5321.25
		Gene birth rate                 = 0.1
		Gene death rate                 = 0.1
		Gene transfer rate              = 0
		Individuals to sample per locus = 1
		Effective pop size per locus    = 1
		Tree fraction to set outgroup   = 0
		Species tree input as newick    = 
		Tree scale                      = -1

Seeds = {52702, 23707}
############################################################
####	treeducken, version 0.1 			####
####	e845a82c08ba308f75f94a270b81a368

In [8]:
%%bash
mkdir -p settings
mkdir -p hsrate_0-0
mkdir -p hsrate_0-25
mkdir -p hsrate_0-5
mkdir -p hsrate_1-0
mv *settings.txt settings/
mv hsrate_0.0* hsrate_0-0/
mv hsrate_0.25* hsrate_0-25/
mv hsrate_0.5* hsrate_0-5/
mv hsrate_1.0* hsrate_1-0/

### Creating extra datafiles for use with DEC model
With trees in hand, I need to make a few more files to run the epoch DEC model. I need to make a "range" file for the symbionts. To do this we need to parse the tips of the locus tree which is our stand-in for the symbiont tree. First, I'm going to set up the R environment to process tree files.

In [10]:
%%bash
pip install --upgrade pip
pip install rpy2

  Running setup.py bdist_wheel for rpy2: started
  Running setup.py bdist_wheel for rpy2: finished with status 'done'
  Stored in directory: /home/jovyan/.cache/pip/wheels/a8/70/ed/8ed99930ef5a7ba56f7bacb08d9363ecd29bc289eb80c41554
Successfully built rpy2


You are using pip version 9.0.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [19]:
%load_ext rpy2.ipython


SyntaxError: invalid syntax (<ipython-input-19-c252d0bcce1d>, line 2)

In [22]:
%%R
install.packages("ape",repos='http://cran.us.r-project.org', quiet=TRUE)
library(ape)

R[write to console]: Updating HTML index of packages in '.Library'

R[write to console]: Making 'packages.html' ...
R[write to console]:  done



I will also need to encode my "paleogeographic" data in terms of the divergence times of the species which define our epochs, the distances defined by the species tree that way we can test whether random vs. clade biased host-switching provides a better fit. We also need connectivity to define which species are available to be hosts at which times.

In [50]:
%%R -i reps
# read in species trees to get the min and max ages of "epochs"
dir_name = "hsrate_0-0/"
prefix_fn = "hsrate_0.0_"
suffix_fn = ".sp.tre"
for (i in 0:0){
    # print out epoch times (no uncertainty in age here)
    fn = paste0(dir_name, prefix_fn, i, suffix_fn)
    tree = read.nexus(file = fn)
    br_times = sort(c(0.0, branching.times(tree)), decreasing = TRUE)
    out_fn = paste0(dir_name, prefix_fn, i, ".times.txt")
    write(br_times, file = out_fn, ncolumns = 1)
    # print out connectivity graphs
    
}

In [ ]:
def nexus_range_prefix_print(num_hosts, num_symbs, file_begin):
    file_name_end = ".range.nex"
    file_begin += file_name_end       
    file_handle = open(file_begin, "w")
    # the header part
    file_handle.write("#NEXUS\n")
    file_handle.write("\n")
    file_handle.write("Begin data;")
    file_handle.write("Dimensions ntax="+str(num_symbs)+" nchar="+str(num_hosts)+"=;")
    file_handle.write("Format datatype=Standard missing=? gap=- labels="01";")
    file_handle.write("Matrix\n")
    
    # the symbionts and associated hosts
    
    
    
    file_handle.write(";\nEnd;")
    file_handle.close()

Finally we just need to write up our Rev scripts and run them. To be done outside of ipython notebook...